In [15]:
import geopandas as gpd
import subprocess
from gelos.config import RAW_DATA_DIR, PROCESSED_DATA_DIR, DATA_VERSION, INTERIM_DATA_DIR
import shutil

In [16]:
# set directory names and creat an app files directory
output_dir = PROCESSED_DATA_DIR / DATA_VERSION
interim_dir = INTERIM_DATA_DIR / DATA_VERSION
interim_dir.mkdir(exist_ok=True, parents=True)
app_files_dir = output_dir / "app_files"
app_files_dir.mkdir(exist_ok=True, parents=True)
# get a list of all embedding csv paths
embedding_csv_paths = output_dir.rglob("*tsne.csv")

In [17]:
# get the original chip tracker from the raw data directory
data_root = RAW_DATA_DIR / DATA_VERSION
chip_gdf = gpd.read_file(data_root / 'gelos_chip_tracker.geojson')

# get only the fields we need for pmtiles
chip_gdf = chip_gdf[[
    "category",
    "sentinel_1_dates",
    "sentinel_2_dates",
    "landsat_dates",
    "id",
    "lat",
    "lon",
    "color",
    "landsat_thumbs",
    "sentinel_1_thumbs",
    "sentinel_2_thumbs",
    "geometry",
    ]]


In [18]:
# for the points.json, drop geometry to make the file smaller and faster to load in the browser
points_df = chip_gdf.drop("geometry", axis=1)
points_df.to_json(app_files_dir / "points.json")

In [19]:
# copy all embedding files to json in the app files directory
for embedding_csv_path in embedding_csv_paths:
    embed_df = gpd.read_file(embedding_csv_path)
    embed_df.to_json(app_files_dir / f"{embedding_csv_path.stem}.json")

In [20]:
# save centroids and chips to temporary files for tippecanoe to load from
chip_gdf_centroids = chip_gdf.copy()
chip_gdf_centroids["geometry"] = chip_gdf_centroids.geometry.centroid
chip_gdf_centroids = chip_gdf_centroids.set_geometry("geometry")
chip_gdf_centroids.to_file(interim_dir / "gelos_app_centroids.geojson", driver="GeoJSON")
chip_gdf.to_file(interim_dir / "gelos_app_chip_tracker.geojson", driver="GeoJSON")

/tmp/ipykernel_814/4187391018.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  chip_gdf_centroids["geometry"] = chip_gdf_centroids.geometry.centroid


In [21]:

# generate 
cmd = f"""
tippecanoe -f -Z5 -z14 \
  -ps \
  --no-tiny-polygon-reduction \
  --no-tile-size-limit \
  --no-feature-limit \
  --drop-densest-as-needed \
  --extend-zooms-if-still-dropping \
  -l gelos_centroids \
  -o {str(app_files_dir / "centroids.pmtiles")} \
  {str(interim_dir / "gelos_app_centroids.geojson")}
"""

subprocess.run(cmd, shell=True, check=True)

cmd = f"""
tippecanoe -f -Z5 -z14 \
  -ps \
  --no-tiny-polygon-reduction \
  --no-tile-size-limit \
  --no-feature-limit \
  --drop-densest-as-needed \
  --extend-zooms-if-still-dropping \
  -l gelos_chips \
  -o {str(app_files_dir / "gelos_chip_tracker.pmtiles")} \
  {str(interim_dir / "gelos_app_chip_tracker.geojson")}
"""


subprocess.run(cmd, shell=True, check=True)

60507 features, 5685049 bytes of geometry and attributes, 57389324 bytes of string pool, 0 bytes of vertices, 0 bytes of nodes
  99.9%  14/3598/5388    
60507 features, 7128697 bytes of geometry and attributes, 57389324 bytes of string pool, 0 bytes of vertices, 0 bytes of nodes
  99.9%  14/15026/9920   
  100.0%  14/9637/3855   

CompletedProcess(args='\ntippecanoe -f -Z5 -z14   -ps   --no-tiny-polygon-reduction   --no-tile-size-limit   --no-feature-limit   --drop-densest-as-needed   --extend-zooms-if-still-dropping   -l gelos_chips   -o /app/data/processed/v0.50.0/app_files/gelos_chip_tracker.pmtiles   /app/data/interim/v0.50.0/gelos_app_chip_tracker.geojson\n', returncode=0)